In [1]:
import numpy as np
import pandas as pd

In [2]:
sentiment = pd.read_csv("data/tweets_sentiment.csv")
sentiment.drop(sentiment.columns[[0]], axis=1, inplace=True)
sentiment.rename(columns = {'created_at':'Date'}, inplace = True)
spy = pd.read_csv("data/SPY.csv")

In [33]:
df = pd.merge(sentiment, spy, on ='Date', how ="inner")
df.head()

,Date,pos_score,neu_score,neg_score,compound,Open,High,Low,Close,Adj Close,Volume
0,2020-04-09,0.073174,0.887866,0.038960,0.114438,277.579987,281.200012,275.470001,278.200012,271.092224,189999200
1,2020-04-13,0.076010,0.887225,0.036771,0.122273,277.140015,277.510010,271.410004,275.660004,268.617126,114839100
2,2020-04-14,0.083978,0.871450,0.044567,0.136125,280.980011,284.899994,275.510010,283.790009,276.539368,134143400
3,2020-04-15,0.083737,0.875221,0.041041,0.152802,277.570007,283.940002,275.459991,277.760010,270.663422,121775000
4,2020-04-16,0.081858,0.878151,0.039988,0.127939,279.149994,280.029999,275.760010,279.100006,271.969208,131798300


In [54]:
opn = {'Date': [df['Date'][i] for i in range(4,len(df))], \
       'Movement': [1 if (df['Open'][i] - df['Open'][i-1]) / (df['Open'][i-1]) > 0.005 else 0 for i in range(4,len(df))], \
       'Move1': [1 if (df['Open'][i-1] - df['Open'][i-2]) / (df['Open'][i-2]) > 0.005 else 0 for i in range(4,len(df))], \
       'Move2': [1 if (df['Open'][i-2] - df['Open'][i-3]) / (df['Open'][i-3]) > 0.005 else 0 for i in range(4,len(df))], \
       'Move3': [1 if (df['Open'][i-3] - df['Open'][i-4]) / (df['Open'][i-4]) > 0.005 else 0 for i in range(4,len(df))]}

sen = {'Date': [df['Date'][i] for i in range(4,len(df))], \
       'Sentiment1': [1 if (df['compound'][i] - df['compound'][i-1]) / (df['compound'][i-1]) > 0.005 else 0  for i in range(4,len(df))], \
       'Sentiment2': [1 if (df['compound'][i-1] - df['compound'][i-2]) / (df['compound'][i-2]) > 0.005 else 0 for i in range(4,len(df))], \
       'Sentiment3': [1 if (df['compound'][i-2] - df['compound'][i-3]) / (df['compound'][i-3]) > 0.005 else 0 for i in range(4,len(df))]}

In [55]:
o = pd.DataFrame(opn)
s = pd.DataFrame(sen)
data = pd.merge(o, s, on='Date', how='inner')

In [56]:
data.head()

,Date,Movement,Move1,Move2,Move3,Sentiment1,Sentiment2,Sentiment3
0,2020-04-16,1,0,1,0,0,1,1
1,2020-04-17,1,1,0,1,1,0,1
2,2020-04-20,0,1,1,0,0,1,0
3,2020-04-21,0,0,1,1,0,0,1
4,2020-04-22,1,0,0,1,1,0,0


## Training Model

In [57]:
from sklearn.model_selection import train_test_split
X = data[data.columns[2:]]
y = data[data.columns[1]]
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3)

In [58]:
positives = sum([1 if y[i] == 1 else 0 for i in range(len(y))])
negatives = len(y) - positives
print("Positive: ", positives)
print("Negative: ", negatives)

Positive:  24
Negative:  25


In [59]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

### Linear Kernel

In [60]:
from sklearn.svm import SVC
classifier = SVC(kernel='linear',verbose=True)
classifier.fit(X_train, y_train)

[LibSVM].


SVC(kernel='linear', verbose=True)

*
optimization finished, #iter = 62
obj = -19.493252, rho = -0.531275
nSV = 24, nBSV = 15
Total nSV = 24


In [61]:
y_pred = classifier.predict(X_test)

In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred));

              precision    recall  f1-score   support

           0       0.45      1.00      0.62         5
           1       1.00      0.40      0.57        10

    accuracy                           0.60        15
   macro avg       0.73      0.70      0.60        15
weighted avg       0.82      0.60      0.59        15



### RBF Kernel

In [63]:
classifier = SVC(kernel='rbf',verbose=True)
classifier.fit(X_train, y_train)

[LibSVM]*
optimization finished, #iter = 33
obj = -19.773839, rho = -0.414048
nSV = 31, nBSV = 18
Total nSV = 31


SVC(verbose=True)

In [64]:
y_pred = classifier.predict(X_test)

In [65]:
print(classification_report(y_test, y_pred));

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         5
           1       1.00      0.50      0.67        10

    accuracy                           0.67        15
   macro avg       0.75      0.75      0.67        15
weighted avg       0.83      0.67      0.67        15



### Polynomial Kernel

In [66]:
classifier = SVC(kernel='poly',verbose=True)
classifier.fit(X_train, y_train)

[LibSVM]

SVC(kernel='poly', verbose=True)

*
optimization finished, #iter = 29
obj = -19.309641, rho = -0.109694
nSV = 29, nBSV = 23
Total nSV = 29


In [67]:
y_pred = classifier.predict(X_test)

In [68]:
print(classification_report(y_test, y_pred));

              precision    recall  f1-score   support

           0       0.22      0.40      0.29         5
           1       0.50      0.30      0.37        10

    accuracy                           0.33        15
   macro avg       0.36      0.35      0.33        15
weighted avg       0.41      0.33      0.35        15



### Sigmoid Kernel

In [69]:
classifier = SVC(kernel='sigmoid',verbose=True)
classifier.fit(X_train, y_train)

[LibSVM].*

SVC(kernel='sigmoid', verbose=True)

.*
optimization finished, #iter = 69
obj = -23.980124, rho = -0.521266
nSV = 30, nBSV = 22
Total nSV = 30


In [70]:
y_pred = classifier.predict(X_test)

In [71]:
print(classification_report(y_test, y_pred));

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         5
           1       1.00      0.50      0.67        10

    accuracy                           0.67        15
   macro avg       0.75      0.75      0.67        15
weighted avg       0.83      0.67      0.67        15

